# Creating first agent

## Install Packages

In [ ]:
%pip install azure-ai-projects
%pip install azure-identity

## Import the libraries

In [13]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [4]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Create the agent

In [55]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_8SgH8JYYYS7uGjXOcQYK3eec


## Create thread

In [56]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_LWDsEDvj8xNQr4HzFAlJJSnW


## Helper function
- adds messages to the thread
- run the agent
- check the run status
- display the agent response

In [57]:
def run_agent(user_input):
  # Add a message to the thread
  message = project_client.agents.messages.create(
      thread_id=thread.id,
      role="user",  # Role of the message sender
      content=user_input,  # Message content
  )
  print(f"Created message, ID: {message['id']}")

  # Create and process a run for the agent to handle the message
  run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)
  print(f"Created run, ID: {run.id}")
      
  # Poll the run status until it is completed or requires action
  while run.status in ["queued", "in_progress", "requires_action"]:
      time.sleep(1)
      run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    
  print(f"Run completed with status: {run.status}")
      
  # Fetch and log all messages from the thread
  messages = project_client.agents.messages.list(thread_id=thread.id)
  for message in messages:
      print(f"Role: {message['role']}, Content: {message['content']}")

## Run the agent

In [58]:
user_input = "What can I see in Manila?"
run_agent(user_input)

Created message, ID: msg_uRxHAmXtJTRy5pVIhMiB8hnO
Created run, ID: run_xmYUbr7d9gqY8sva0xdBiJTz
Run completed with status: completed
Role: assistant, Content: [{'type': 'text', 'text': {'value': 'Manila, the capital city of the Philippines, is a vibrant metropolis with a rich history and diverse culture. Here are some key attractions you can visit in Manila:\n\n1. **Rizal Park (Luneta Park)**: A historic park dedicated to the national hero, Dr. José Rizal. It features various monuments, gardens, and a large open space for picnics and outdoor activities.\n\n2. **Intramuros**: Known as the "Walled City," Intramuros is the oldest district in Manila, featuring colonial-era architecture, historical landmarks, and museums. Notable sites include Fort Santiago, San Agustin Church, and Casa Manila.\n\n3. **Manila Cathedral**: An impressive Roman Catholic basilica located within Intramuros, with stunning architecture and historical significance.\n\n4. **National Museum of the Philippines**: Comp

In [59]:
user_input = "How about New York?"
run_agent(user_input)

Created message, ID: msg_aWkFBZUCfwrhVEmYhEHzr1Rj
Created run, ID: run_o33uul7W2bouD3jhiuGRLnpU
Run completed with status: completed
Role: assistant, Content: [{'type': 'text', 'text': {'value': "New York City is one of the most iconic cities in the world, known for its vibrant culture, diverse neighborhoods, and numerous landmarks. Here are some key attractions you can visit in New York City:\n\n### Manhattan\n1. **Times Square**: The bustling heart of Midtown Manhattan, famous for its bright lights, Broadway theaters, and vibrant atmosphere.\n2. **Central Park**: An expansive urban park offering various attractions, including the Central Park Zoo, Bethesda Terrace, and numerous walking trails.\n3. **Statue of Liberty and Ellis Island**: Iconic symbols of freedom, accessible by ferry, offering insights into America's immigrant history.\n4. **Empire State Building**: One of the most famous skyscrapers in the world, offering stunning views from its observation decks.\n5. **Rockefeller C

In [62]:
user_input = "How about San Francisco?"
run_agent(user_input)

Created message, ID: msg_6HuzxKAgpfTduvnFFu2ioGEh
Created run, ID: run_JASBH5tJxKQ7rViPL3R0MN0h
Run completed with status: failed
Role: user, Content: [{'type': 'text', 'text': {'value': 'How about San Francisco?', 'annotations': []}}]
Role: user, Content: [{'type': 'text', 'text': {'value': 'How about San Francisco?', 'annotations': []}}]
Role: user, Content: [{'type': 'text', 'text': {'value': 'How about San Francisco?', 'annotations': []}}]
Role: assistant, Content: [{'type': 'text', 'text': {'value': "New York City is one of the most iconic cities in the world, known for its vibrant culture, diverse neighborhoods, and numerous landmarks. Here are some key attractions you can visit in New York City:\n\n### Manhattan\n1. **Times Square**: The bustling heart of Midtown Manhattan, famous for its bright lights, Broadway theaters, and vibrant atmosphere.\n2. **Central Park**: An expansive urban park offering various attractions, including the Central Park Zoo, Bethesda Terrace, and numer

## Cleanup resources

In [54]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
